In [1]:
#!/usr/bin/env python2
from __future__ import print_function
import os, sys, time, copy

import random
import numpy as np
# import matplotlib
# matplotlib.use('Agg')
import matplotlib.pyplot as plt
from multiprocessing import freeze_support
import pints#!/usr/bin/env python
import bisect

sys.path.append('../')
import model_ikr as m
import parametertransform
from protocols import leak_staircase as protocol_def
from priors import prior_parameters
from priors_JK import BeattieLogPrior as LogPrior

sys.path.append('../../../')
sys.path.append('../../../Protocols')
sys.path.append('../../../Models')
sys.path.append('../../../Lib')
import protocol_lib
import vc_protocols


random.seed(542811797)
np.random.seed(542811797)
        
# Parameter range setting
dataset_dir = 'Kylie'
parameter_ranges = []
if 'Kylie' in dataset_dir:    
    if 'rmax600' in dataset_dir:
        # Kylie: rmax = 600 
        parameter_ranges = []
        parameter_ranges.append( [100, 500000] )
        parameter_ranges.append( [0.0001, 598] )
        parameter_ranges.append( [0.0001, 260] )
        parameter_ranges.append( [0.0001, 598] )
        parameter_ranges.append( [0.0001, 130] )
        parameter_ranges.append( [0.0001, 598] )
        parameter_ranges.append( [0.0001, 260] )
        parameter_ranges.append( [0.0001, 598] )
        parameter_ranges.append( [0.0001, 130] )
        print("Kylie-rmax600 dataset has been selected.")
    else :
        # Kylie
        parameter_ranges.append([100, 500000])
        parameter_ranges.append( [0.0001, 1000000])
        parameter_ranges.append( [0.0001, 384])
        parameter_ranges.append( [0.0001, 1000000] )
        parameter_ranges.append( [0.0001, 192] )
        parameter_ranges.append( [0.0001, 1000000] )
        parameter_ranges.append( [0.0001, 384] )
        parameter_ranges.append( [0.0001, 1000000] )
        parameter_ranges.append( [0.0001, 192] )
        print("Kylie dataset has been selected.")

elif 'RealRange' in dataset_dir:
        parameter_ranges.append([3134, 500000])                 # g
        parameter_ranges.append( [0.0001, 2.6152843264828003])  # p1
        parameter_ranges.append( [43.33271226094526, 259])      # p2
        parameter_ranges.append( [0.001, 0.5] )                 # p3
        parameter_ranges.append( [15, 75] )                     # p4
        parameter_ranges.append( [0.8, 410] )                   # p5
        parameter_ranges.append( [0.0001, 138.] )               # p6
        parameter_ranges.append( [1.0, 59] )                    # p7
        parameter_ranges.append( [1.6, 90] )                    # p8
        print("RealRange dataset has been selected.")

parameter_ranges = np.array(parameter_ranges)
print(parameter_ranges.shape)

 
param_names = ['g', 'p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8']


# Mean Squared Error
def MSE(y, y_pred, axis=1):
	return np.mean(np.square(y - y_pred), axis=axis)

Kylie dataset has been selected.
(9, 2)
Kylie dataset has been selected.
(9, 2)


In [2]:
a= np.ones(10)
b = np.zeros(10)
MSE(a,b, axis=0)

1.0

In [3]:
import cma
# help(cma)

# Set up simulation
options = cma.CMAOptions()

# Tell cma-es to be quiet
options.set('verbose', -9)

# Set population size
options.set('popsize', 4)

# CMAES always seeds np.random, whether you ask it too or not, so to
# get consistent debugging output, we should always pass in a seed.
# Instead of using a fixed number (which would be bad), we can use a
# randomly generated number: This will ensure pseudo-randomness, but
# produce consistent results if np.random has been seeded before
# calling.
options.set('seed', np.random.randint(2**31))

es = cma.CMAEvolutionStrategy([1,2], 0.01, options)

xs = np.array(es.ask())
print(xs)
es.tell(xs, [1,2,3,4])
print(es.result.xbest)
print(es.result.fbest)
print("="*10)
xs = np.array(es.ask())
print(xs)
es.tell(xs, [30,40,5,6])
print(es.result.xbest)
print(es.result.fbest)
print("="*10)
xs = np.array(es.ask())
print(xs)
es.tell(xs, [30,40,5,6])
print(es.result.xbest)
print(es.result.fbest)


# es.optimize(cma.ff.rosen)
# es.result_pretty()  # pretty print result

[[0.99191527 1.976241  ]
 [1.01057657 2.0156819 ]
 [1.01204235 2.01049521]
 [0.99050006 1.98470138]]
[0.99191527 1.976241  ]
1
[[1.00172657 1.99387972]
 [0.99505808 2.01205833]
 [0.9907601  2.00011739]
 [0.98450059 1.99546681]]
[0.99191527 1.976241  ]
1
[[0.98993253 1.97734342]
 [0.99260507 2.00333557]
 [0.98931992 1.99744514]
 [0.99271165 1.98640734]]
[0.99191527 1.976241  ]
1


In [4]:
class OptimisationController(object):
    """
    Finds the parameter values that minimise an :class:`ErrorMeasure` or
    maximise a :class:`LogPDF`.

    Parameters
    ----------
    function
        An :class:`pints.ErrorMeasure` or a :class:`pints.LogPDF` that
        evaluates points in the parameter space.
    x0
        The starting point for searches in the parameter space. This value may
        be used directly (for example as the initial position of a particle in
        :class:`PSO`) or indirectly (for example as the center of a
        distribution in :class:`XNES`).
    sigma0
        An optional initial standard deviation around ``x0``. Can be specified
        either as a scalar value (one standard deviation for all coordinates)
        or as an array with one entry per dimension. Not all methods will use
        this information.
    boundaries
        An optional set of boundaries on the parameter space.
    transformation
        An optional :class:`pints.Transformation` to allow the optimiser to
        search in a transformed parameter space. If used, points shown or
        returned to the user will first be detransformed back to the original
        space.
    method
        The class of :class:`pints.Optimiser` to use for the optimisation.
        If no method is specified, :class:`CMAES` is used.
    """

    def __init__(
            self, function, x0, sigma0=None, boundaries=None,
            transformation=None, method=None):

        # Convert x0 to vector
        # This converts e.g. (1, 7) shapes to (7, ), giving users a bit more
        # freedom with the exact shape passed in. For example, to allow the
        # output of LogPrior.sample(1) to be passed in.
        x0 = pints.vector(x0)

        # Check dimension of x0 against function
        if function.n_parameters() != len(x0):
            raise ValueError(
                'Starting point must have same dimension as function to'
                ' optimise.')

        # Check if minimising or maximising
        self._minimising = not isinstance(function, pints.LogPDF)

        # Apply a transformation (if given). From this point onward the
        # optimiser will see only the transformed search space and will know
        # nothing about the model parameter space.
        if transformation is not None:
            # Convert error measure or log pdf
            if self._minimising:
                function = transformation.convert_error_measure(function)
            else:
                function = transformation.convert_log_pdf(function)

            # Convert initial position
            x0 = transformation.to_search(x0)

            # Convert sigma0, if provided
            if sigma0 is not None:
                sigma0 = transformation.convert_standard_deviation(sigma0, x0)
            if boundaries:
                boundaries = transformation.convert_boundaries(boundaries)

        # Store transformation for later detransformation: if using a
        # transformation, any parameters logged to the filesystem or printed to
        # screen should be detransformed first!
        self._transformation = transformation

        # Store function
        if self._minimising:
            self._function = function
        else:
            self._function = pints.ProbabilityBasedError(function)
        del(function)

        # Create optimiser
        if method is None:
            method = pints.CMAES
        elif not issubclass(method, pints.Optimiser):
            raise ValueError('Method must be subclass of pints.Optimiser.')
        self._optimiser = method(x0, sigma0, boundaries)

        # Check if sensitivities are required
        self._needs_sensitivities = self._optimiser.needs_sensitivities()

        # Logging
        self._log_to_screen = True
        self._log_filename = None
        self._log_csv = False
        self.set_log_interval()

        # Parallelisation
        self._parallel = False
        self._n_workers = 1
        self.set_parallel()

        # User callback
        self._callback = None

        # :meth:`run` can only be called once
        self._has_run = False

        #
        # Stopping criteria
        #

        # Maximum iterations
        self._max_iterations = None
        self.set_max_iterations()

        # Maximum unchanged iterations
        self._max_unchanged_iterations = None
        self._min_significant_change = 1
        self.set_max_unchanged_iterations()

        # Threshold value
        self._threshold = None

        # Post-run statistics
        self._evaluations = None
        self._iterations = None
        self._time = None
        
        self.update_record = []

    def evaluations(self):
        """
        Returns the number of evaluations performed during the last run, or
        ``None`` if the controller hasn't ran yet.
        """
        return self._evaluations


    def iterations(self):
        """
        Returns the number of iterations performed during the last run, or
        ``None`` if the controller hasn't ran yet.
        """
        return self._iterations


    def max_iterations(self):
        """
        Returns the maximum iterations if this stopping criterion is set, or
        ``None`` if it is not. See :meth:`set_max_iterations()`.
        """
        return self._max_iterations


    def max_unchanged_iterations(self):
        """
        Returns a tuple ``(iterations, threshold)`` specifying a maximum
        unchanged iterations stopping criterion, or ``(None, None)`` if no such
        criterion is set. See :meth:`set_max_unchanged_iterations()`.
        """
        if self._max_unchanged_iterations is None:
            return (None, None)
        return (self._max_unchanged_iterations, self._min_significant_change)


    def optimiser(self):
        """
        Returns the underlying optimiser object, allowing detailed
        configuration.
        """
        return self._optimiser


    def parallel(self):
        """
        Returns the number of parallel worker processes this routine will be
        run on, or ``False`` if parallelisation is disabled.
        """
        return self._n_workers if self._parallel else False


    def run(self):
        """
        Runs the optimisation, returns a tuple ``(xbest, fbest)``.

        An optional ``callback`` function can be passed in that will be called
        at the end of every iteration. The callback should take the arguments
        ``(iteration, optimiser)``, where ``iteration`` is the iteration count
        (an integer) and ``optimiser`` is the optimiser object.
        """
        # Can only run once for each controller instance
        if self._has_run:
            raise RuntimeError("Controller is valid for single use only")
        self._has_run = True

        # Check stopping criteria
        has_stopping_criterion = False
        has_stopping_criterion |= (self._max_iterations is not None)
        has_stopping_criterion |= (self._max_unchanged_iterations is not None)
        has_stopping_criterion |= (self._threshold is not None)
        if not has_stopping_criterion:
            raise ValueError('At least one stopping criterion must be set.')

        # Iterations and function evaluations
        iteration = 0
        evaluations = 0

        # Unchanged iterations count (used for stopping or just for
        # information)
        unchanged_iterations = 0

        # Choose method to evaluate
        f = self._function
        if self._needs_sensitivities:
            f = f.evaluateS1

        # Create evaluator object
        if self._parallel:
            # Get number of workers
            n_workers = self._n_workers

            # For population based optimisers, don't use more workers than
            # particles!
            if isinstance(self._optimiser, pints.PopulationBasedOptimiser):
                n_workers = min(n_workers, self._optimiser.population_size())
            evaluator = pints.ParallelEvaluator(f, n_workers=n_workers)
        else:
            evaluator = pints.SequentialEvaluator(f)

        # Keep track of best position and score
        xbest = self._optimiser._x0
        fbest = f(self._optimiser._x0) #float('inf')        
        print('fbest', fbest)

        # Internally we always minimise! Keep a 2nd value to show the user
        fbest_user = fbest if self._minimising else -fbest

        # Set up progress reporting
        next_message = 0

        # Start logging
        logging = self._log_to_screen or self._log_filename
        if logging:
            if self._log_to_screen:
                # Show direction
                if self._minimising:
                    print('Minimising error measure')
                else:
                    print('Maximising LogPDF')

                # Show method
                print('Using ' + str(self._optimiser.name()))

                # Show parallelisation
                if self._parallel:
                    print('Running in parallel with ' + str(n_workers) +
                          ' worker processes.')
                else:
                    print('Running in sequential mode.')

            # Show population size
            pop_size = 1
            if isinstance(self._optimiser, pints.PopulationBasedOptimiser):
                pop_size = self._optimiser.population_size()
                if self._log_to_screen:
                    print('Population size: ' + str(pop_size))

            # Set up logger
            logger = pints.Logger()
            if not self._log_to_screen:
                logger.set_stream(None)
            if self._log_filename:
                logger.set_filename(self._log_filename, csv=self._log_csv)

            # Add fields to log
            max_iter_guess = max(self._max_iterations or 0, 10000)
            max_eval_guess = max_iter_guess * pop_size
            logger.add_counter('Iter.', max_value=max_iter_guess)
            logger.add_counter('Eval.', max_value=max_eval_guess)
            logger.add_float('Best')
            self._optimiser._log_init(logger)
            logger.add_time('Time m:s')

        # Start searching
        timer = pints.Timer()
        running = True        
        nIter_update = 0        
        try:
            
            while running:
                # Get points
                xs = self._optimiser.ask()
                
                # Calculate scores
                fs = evaluator.evaluate(xs)
              
                # Perform iteration
                self._optimiser.tell(fs)

                # Check if new best found                
                fnew = self._optimiser.fbest()
                         
                if fnew < fbest:                    
                    # Check if this counts as a significant change
                    if np.abs(fnew - fbest) < self._min_significant_change:
                        unchanged_iterations += 1
                    else:
                        unchanged_iterations = 0

                    # Update best
                    xbest = self._optimiser.xbest()
                    fbest = fnew

                    # Update user value of fbest
                    fbest_user = fbest if self._minimising else -fbest
                    
                    nIter_update += 1       
                    self.update_record.append((iteration+1, nIter_update))
#                     print("Epoch : %d  |  nUpdate : %d"%(iteration+1, nIter_update))     
                else:
                    unchanged_iterations += 1
                    
                                
                # Update evaluation count
                evaluations += len(fs)

                # Show progress
                if logging and iteration >= next_message:
                    # Log state
                    logger.log(iteration, evaluations, fbest_user)
                    self._optimiser._log_write(logger)
                    logger.log(timer.time())

                    # Choose next logging point
                    if iteration < self._message_warm_up:
                        next_message = iteration + 1
                    else:
                        next_message = self._message_interval * (
                            1 + iteration // self._message_interval)

                # Update iteration count
                iteration += 1

                #
                # Check stopping criteria
                #

                # Maximum number of iterations
                if (self._max_iterations is not None and
                        iteration >= self._max_iterations):
                    running = False
                    halt_message = ('Halting: Maximum number of iterations ('
                                    + str(iteration) + ') reached.')

                # Maximum number of iterations without significant change
                halt = (self._max_unchanged_iterations is not None and
                        unchanged_iterations >= self._max_unchanged_iterations)
                if halt:
                    running = False
                    halt_message = ('Halting: No significant change for ' +
                                    str(unchanged_iterations) + ' iterations.')

                # Threshold value
                if self._threshold is not None and fbest < self._threshold:
                    running = False
                    halt_message = ('Halting: Objective function crossed'
                                    ' threshold: ' + str(self._threshold) +
                                    '.')

                # Error in optimiser
                error = self._optimiser.stop()
                if error:   # pragma: no cover
                    running = False
                    halt_message = ('Halting: ' + str(error))

                elif self._callback is not None:
                    self._callback(iteration - 1, self._optimiser)
                                       
        except (Exception, SystemExit, KeyboardInterrupt):  # pragma: no cover
            # Unexpected end!
            # Show last result and exit
            print('\n' + '-' * 40)
            print('Unexpected termination.')
            print('Current best score: ' + str(fbest))
            print('Current best position:')

            # Inverse transform search parameters
            if self._transformation:
                xbest = self._transformation.to_model(xbest)
#             else:
#                 xbest = xbest

            for p in xbest:
                print(pints.strfloat(p))
            print('-' * 40)
            raise

        # Stop timer
        self._time = timer.time()

        # Log final values and show halt message
        if logging:
            logger.log(iteration, evaluations, fbest_user)
            self._optimiser._log_write(logger)
            logger.log(self._time)
            if self._log_to_screen:
                print(halt_message)

        # Save post-run statistics
        self._evaluations = evaluations
        self._iterations = iteration

        # Inverse transform search parameters
        if self._transformation:
            xbest = self._transformation.to_model(xbest)
#         else:
#             xbest = xbest
        
        # Return best position and score
        return xbest, fbest_user


    def set_callback(self, cb=None):
        """
        Allows a "callback" function to be passed in that will be called at the
        end of every iteration.

        This can be used for e.g. visualising optimiser progress.

        Example::

            def cb(opt):
                plot(opt.xbest())

            opt.set_callback(cb)

        """
        if cb is not None and not callable(cb):
            raise ValueError('The argument cb must be None or a callable.')
        self._callback = cb


    def set_log_interval(self, iters=20, warm_up=3):
        """
        Changes the frequency with which messages are logged.

        Parameters
        ----------
        ``interval``
            A log message will be shown every ``iters`` iterations.
        ``warm_up``
            A log message will be shown every iteration, for the first
            ``warm_up`` iterations.
        """
        iters = int(iters)
        if iters < 1:
            raise ValueError('Interval must be greater than zero.')
        warm_up = max(0, int(warm_up))

        self._message_interval = iters
        self._message_warm_up = warm_up


    def set_log_to_file(self, filename=None, csv=False):
        """
        Enables logging to file when a filename is passed in, disables it if
        ``filename`` is ``False`` or ``None``.

        The argument ``csv`` can be set to ``True`` to write the file in comma
        separated value (CSV) format. By default, the file contents will be
        similar to the output on screen.
        """
        if filename:
            self._log_filename = str(filename)
            self._log_csv = True if csv else False
        else:
            self._log_filename = None
            self._log_csv = False


    def set_log_to_screen(self, enabled):
        """
        Enables or disables logging to screen.
        """
        self._log_to_screen = True if enabled else False


    def set_max_iterations(self, iterations=10000):
        """
        Adds a stopping criterion, allowing the routine to halt after the
        given number of ``iterations``.

        This criterion is enabled by default. To disable it, use
        ``set_max_iterations(None)``.
        """
        if iterations is not None:
            iterations = int(iterations)
            if iterations < 0:
                raise ValueError(
                    'Maximum number of iterations cannot be negative.')
        self._max_iterations = iterations


    def set_max_unchanged_iterations(self, iterations=200, threshold=1e-11):
        """
        Adds a stopping criterion, allowing the routine to halt if the
        objective function doesn't change by more than ``threshold`` for the
        given number of ``iterations``.

        This criterion is enabled by default. To disable it, use
        ``set_max_unchanged_iterations(None)``.
        """
        if iterations is not None:
            iterations = int(iterations)
            if iterations < 0:
                raise ValueError(
                    'Maximum number of iterations cannot be negative.')

        threshold = float(threshold)
        if threshold < 0:
            raise ValueError('Minimum significant change cannot be negative.')

        self._max_unchanged_iterations = iterations
        self._min_significant_change = threshold


    def set_parallel(self, parallel=False):
        """
        Enables/disables parallel evaluation.

        If ``parallel=True``, the method will run using a number of worker
        processes equal to the detected cpu core count. The number of workers
        can be set explicitly by setting ``parallel`` to an integer greater
        than 0.
        Parallelisation can be disabled by setting ``parallel`` to ``0`` or
        ``False``.
        """
        if parallel is True:
            self._parallel = True
            self._n_workers = pints.ParallelEvaluator.cpu_count()
        elif parallel >= 1:
            self._parallel = True
            self._n_workers = int(parallel)
        else:
            self._parallel = False
            self._n_workers = 1


    def set_threshold(self, threshold):
        """
        Adds a stopping criterion, allowing the routine to halt once the
        objective function goes below a set ``threshold``.

        This criterion is disabled by default, but can be enabled by calling
        this method with a valid ``threshold``. To disable it, use
        ``set_treshold(None)``.
        """
        if threshold is None:
            self._threshold = None
        else:
            self._threshold = float(threshold)


    def threshold(self):
        """
        Returns the threshold stopping criterion, or ``None`` if no threshold
        stopping criterion is set. See :meth:`set_threshold()`.
        """
        return self._threshold


    def time(self):
        """
        Returns the time needed for the last run, in seconds, or ``None`` if
        the controller hasn't run yet.
        """
        return self._time

In [5]:
# sys.path.append(f'../../../../Dataset/herg25oc1_real_data')
# from getdata import get_dataset, get_times

# # for i in range(5):
# # xs, ys = get_dataset(file_numbers=range(1, 3), multi=True, use_torch=True)

# # xs, ys = get_dataset(file_numbers=range(1, 31), window=10, step_size=5, multi=False, use_torch=False, get_raw=False)

# cell_to_remove=['A04', 'A15', 'E05', 'E22', 'G10', 'G22', 'H08', 'I10', 'I12', 'I16', 'L08', 'P15', 'P18' ] # M24
# cell_to_remove=[]

# xs, ys, cells = get_dataset(cell_to_remove=cell_to_remove, scale=1, multi=True, torch_tensor=False)  # <-- fast
# 

In [6]:
def get_modified_current( params, current ):  
    
    times = params['time']
    vc_protocol = params['vc']    
    neighbor_radius = params['neighbor_radius']
    
    method = params['method']
         
    pre_step = vc_protocol.steps[0]
    pre_step_index = 0
    modified_current = copy.copy(current)
    
       
    for i, t in enumerate(times):
        """Gets the voltage based on provided steps for the specified time."""
        # pre_V = vc_protocol.get_voltage_at_time(times[i-1])
        V = vc_protocol.get_voltage_at_time(t)
                        
        step_index = bisect.bisect_left( vc_protocol.get_voltage_change_endpoints(), t )
        current_step = vc_protocol.steps[step_index]
        if (pre_step_index != step_index) :
                        
            neighboring_value = current[i-10:i-1].mean()
                
            if method == 0:
                modified_current[i] = neighboring_value                    
                for j in range(1, neighbor_radius+1):
                    left_neighbor_index = i - j 
                    right_neighbor_index = i + j                 
                    modified_current[left_neighbor_index] = neighboring_value                                    
                    modified_current[right_neighbor_index] = neighboring_value                    
            elif method == 1:
                if V - vc_protocol.get_voltage_at_time(times[i-1]) > 0 :
                    for j in range( 2*neighbor_radius+1):
                        neighbor_index = i - neighbor_radius + j                     
                        if (current[neighbor_index] - neighboring_value) < -10:
                            modified_current[neighbor_index] = neighboring_value                                    
                elif V - vc_protocol.get_voltage_at_time(times[i-1]) < 0 :
                    for j in range( 2*neighbor_radius+1):
                        neighbor_index = i - neighbor_radius + j                     
                        if (current[neighbor_index] - neighboring_value) > 10:
                            modified_current[neighbor_index] = neighboring_value  

            pre_step_index = step_index
            pre_step = current_step
            
    return modified_current

In [7]:
# VC_protocol = vc_protocols_for_myokit.leak_staircase()
VC_protocol = vc_protocols.leak_staircase()

print(f'The protocol is {VC_protocol.get_voltage_change_endpoints()[-1]} s')

The protocol is 15.4 s


In [10]:
if __name__=='__main__':
    freeze_support()
    
    noise_remove = False
    
    
    savedir = './out'
    if not os.path.isdir(savedir):
        os.makedirs(savedir)
    
    data_dir = '../../../../Dataset/herg25oc1_real_data' #-sweep2'
    qc_dir = '../qc'
    file_list = ['herg25oc1']
    temperatures = [25.0]
    useFilterCap = False
    
    # Control fitting seed --> OR DONT
    # fit_seed = np.random.randint(0, 2**30)
    fit_seed = 542811797
    print('Using seed: ', fit_seed)
    np.random.seed(fit_seed)
    
    # Set parameter transformation
#     transform_to_model_param = parametertransform.log_transform_to_model_param       # return np.exp(out)
#     transform_from_model_param = parametertransform.log_transform_from_model_param   # return np.log(out)
    transform_to_original = parametertransform.transform_to_original       # return np.exp(out)
    transform_to_norm = parametertransform.transform_to_norm    # return np.log(out) 
    
    nCPU = pints.ParallelEvaluator.cpu_count()
    print("The number of process :", nCPU)
    
       
    
    
    for i_file, (file_name, temperature) in enumerate(zip(file_list, temperatures)):
        # Load QC
        # selectedfile = '%s/selected-%s.txt' % (qc_dir, file_name)
        # selectedwell = []
        # with open(selectedfile, 'r') as f:
        #     for l in f:
        #         if not l.startswith('#'):
        #             selectedwell.append(l.split()[0])
    
        # Split each file_name as a separate output dir
        savefolder = file_name# + '-sweep2'
        if not os.path.isdir('%s/%s' % (savedir, savefolder)):
            os.makedirs('%s/%s' % (savedir, savefolder))
            
            
            
        init_param_li = [ 'prior' ]  # prior  |  real  |  predicted  |  random                           
#         cell_li = [
#                     'A06', 'A09', 'A13', 'A17', 'A20', 'A23',
#                     'B03', 'B05', 'B08', 'B11', 'B15', 'B18', 'B20', 
#                     'C01', 'C03', 'C07', 'C10', 'C14', 'C17', 'C20', 'C22',
#                     'D02', 'D04', 'D06', 'D09', 'D17', 'D19', 'D24', 
#                     'E03', 'E06', 'E10', 'E15', 'E18', 'E21', 
#                     'F06', 'F11', 'F14', 'F18',
#                     'G01', 'G03', 'G08', 'G13', 'G21', 
#                     'H03', 'H10', 'H12', 'H16', 'H19', 'H22', 
#                     'I01', 'I04', 'I06', 'I13', 'I17', 'I20', 'I22', 'I24', 
#                     'J02', 'J04', 'J12', 'J14', 'J17', 'J22', 'J24', 
#                     'K03', 'K05', 'K08', 'K11', 'K14', 'K17', 'K22', 
#                     'L01', 'L06', 'L16', 'L22', 
#                     'M05', 'M08', 'M11', 'M16', 'M20', 
#                     'N03', 'N07', 'N10', 'N13', 'N18', 'N24', 
#                     'O04', 'O09', 'O11', 'O15', 'O19', 'O24', 
#                     'P03', 'P08', 'P11', 'P14', 'P19', 'P22'
#                   ]
        cell_li = [
                    'A06'
                  ]
        # file_list = os.listdir(data_dir)
        # param_list = [file for file in file_list if file.endswith("542811797.txt")]
        # for file in param_list:
        #     cell_li.append(file[24:27]) 
        # cell_li = cell_li[200:]
        # print(cell_li)
        
        sigma0 = None
        
        # Load model
        model = m.Model('../../../mmt-model-files/kylie-2017-IKr.mmt',
                        protocol_def=protocol_def,
                        temperature=273.15 + temperature,  # K
                        transform=transform_to_original,
                        useFilterCap=useFilterCap)  # ignore capacitive spike
        
        time_file_name = file_name + '-staircaseramp-times.csv'
        times = np.loadtxt(data_dir + '/' + time_file_name,
                                delimiter=',', skiprows=1) # headers
        
        print("dt :", times[1])
        
        for cell in cell_li:        
            print("Parameter fitting for %s"%(cell))
            
            # Load data file names
            data_file_name = file_name + '-staircaseramp-' + cell + '.csv'
            
            # Save name
            saveas = data_file_name[:-4]
            # print(data_file_name)
            # print(saveas)
            if useFilterCap:
                saveas += '-fcap'                
    
            # Load data
            data = np.loadtxt(data_dir + '/' + data_file_name, delimiter=',', skiprows=1) # headers
            
            if noise_remove:
                params_noise = {
                    'time' : times,
                    'vc' : VC_protocol,
                    'neighbor_radius' : 5,
                    'method' : 1
                }
                data = get_modified_current(params_noise, data)
                        
                    
            noise_sigma = np.std(data[:500])
            print('Estimated noise level: ', noise_sigma)
            print('sigma0: ', sigma0)
                                     
            print(data.shape)
            print(times.shape)
                             
            
            if useFilterCap:
                # Apply capacitance filter to data
                data = data * model.cap_filter(times)
    
            # Create Pints stuffs
            problem = pints.SingleOutputProblem(model, times, data)
            #loglikelihood = pints.KnownNoiseLogLikelihood(problem, noise_sigma)
            loglikelihood = pints.GaussianKnownSigmaLogLikelihood(problem, noise_sigma)
            logprior = LogPrior(transform_to_original, transform_to_norm)
            logposterior = pints.LogPosterior(loglikelihood, logprior)
            
            # MSE
            error_mse = pints.MeanSquaredError(problem) # MeanSquaredError RootMeanSquaredError  NormalisedRootMeanSquaredError
        
            # prior param  
            priorparams = np.asarray(prior_parameters['23.0']  )          
#             priorparams = np.array([26086.585617, 0.276096, 64.486973, 0.019522, 49.343747, 107.503522, 8.068859, 6.581754, 31.552841])
            transformed_prior_params = transform_to_norm(priorparams)    # return np.log(out)
                       
            # real parameters 
            params_file_name = file_name + '-staircaseramp-' + cell + '-solution-' + str(fit_seed) +'.txt'
            real_params = np.loadtxt(data_dir + '/' + params_file_name) # headers
            transformed_real_params = transform_to_norm(np.asarray(real_params)) 
                        
            for init_param in init_param_li:
                start_time = time.time()
                
                eval_metric = None   
                x0 = None
                transformed_x0 = None
                                
                if init_param == 'prior':                               
                    eval_metric = logposterior                    
                    x0 = priorparams
                    transformed_x0 = transformed_prior_params
                                    
                elif init_param == 'real':
                    eval_metric = logposterior  # error_mse    logposterior
                    x0 = real_params                    
                    transformed_x0 = transformed_real_params
                                         
                elif init_param == 'predicted':   
                    params_file_name = 'predicted_params-' + cell + '.txt'                     
                    predicted_params = np.loadtxt('./predicted_params_efficientNet6_Kylie/' + params_file_name) # headers                    
                    
                    predicted_params = np.array([29427.030528, 0.026970, 125.924017, 0.060602, 41.221571, 1461.519111, 39.967293, 50.841402, 21.259713])
                    
                    eval_metric = logposterior  # error_mse    logposterior
                    x0 = predicted_params                    
                    transformed_x0 = transform_to_norm(np.asarray(predicted_params))    # return np.log(out)                    
                    
                elif init_param == 'random':    
                    eval_metric = logposterior                    
                    random.seed(None)
                    np.random.seed(None)
                    
                    # transform_params = logprior.sample()                      
                    x0 = logprior.sample_without_inv_transform()
                    transformed_x0 = transform_to_norm(init_param) 
                    
                    # ikr_g = random.uniform(parameter_ranges[0][0], parameter_ranges[0][1])
                    # ikr_p1 = random.uniform(parameter_ranges[1][0], parameter_ranges[1][1])
                    # ikr_p2 = random.uniform(parameter_ranges[2][0], parameter_ranges[2][1])
                    # ikr_p3 = random.uniform(parameter_ranges[3][0], parameter_ranges[3][1])
                    # ikr_p4 = random.uniform(parameter_ranges[4][0], parameter_ranges[4][1])
                    # ikr_p5 = random.uniform(parameter_ranges[5][0], parameter_ranges[5][1])
                    # ikr_p6 = random.uniform(parameter_ranges[6][0], parameter_ranges[6][1])
                    # ikr_p7 = random.uniform(parameter_ranges[7][0], parameter_ranges[7][1])
                    # ikr_p8 = random.uniform(parameter_ranges[8][0], parameter_ranges[8][1])
                    # ikr_g = 0.5 * (parameter_ranges[0][0] + parameter_ranges[0][1])
                    # ikr_p1 = 0.5 * (parameter_ranges[1][0] + parameter_ranges[1][1])
                    # ikr_p2 = 0.5 * (parameter_ranges[2][0] + parameter_ranges[2][1])
                    # ikr_p3 = 0.5 * (parameter_ranges[3][0] + parameter_ranges[3][1])
                    # ikr_p4 = 0.5 * (parameter_ranges[4][0] + parameter_ranges[4][1])
                    # ikr_p5 = 0.5 * (parameter_ranges[5][0] + parameter_ranges[5][1])
                    # ikr_p6 = 0.5 * (parameter_ranges[6][0] + parameter_ranges[6][1])
                    # ikr_p7 = 0.5 * (parameter_ranges[7][0] + parameter_ranges[7][1])
                    # ikr_p8 = 0.5 * (parameter_ranges[8][0] + parameter_ranges[8][1])
                    # init_param = [ikr_g, ikr_p1, ikr_p2, ikr_p3, ikr_p4, ikr_p5, ikr_p6, ikr_p7, ikr_p8]
                    # transform_params = transform_from_model_param(np.asarray(random_parameters))    # return np.log(out)
                    
                    np.random.seed(fit_seed)
                    
                else:
                    print("No initial parameters has been selected.")
                
                mse = MSE(x0, real_params, axis=0)                       
                mse_norm = MSE(transformed_x0, transformed_real_params, axis=0)
                print("="*50)
                print("Real parameters :", real_params)   
                print("-----%s---------------"%(init_param))  
                print('x0 :', x0)
                print("transformed x0 :", transformed_x0)
                print("MSE :", mse)
                print("MSE for norm :", mse_norm)
                print("="*50)
                
                for _ in range(5):
                    assert(eval_metric(transformed_x0) ==\
                            eval_metric(transformed_x0))
                
#                 try:
#                     N = int(sys.argv[1])
#                 except IndexError:
#                     N = 4
        
                # for i in range(N):
                                    
                print('Starting point : ', x0)
                print('Starting transformed point : ', transformed_x0)
                print('Starting eval_metric : ', eval_metric(transformed_x0))
                #opt = pints.Optimisation(logposterior, x0.T, method=pints.CMAES) # PSO  CMAES  GradientDescent  XNES  NelderMead   
               # opt = pints.OptimisationController(error_mse, x0, method=pints.CMAES) # logposterior | error_mse
            
                opt = OptimisationController(eval_metric, transformed_x0.T, sigma0=None, method=pints.CMAES)
                population_size = os.cpu_count()
                max_unchanged_iter = 50
                unchanged_iter_threshold = 1e-11
                opt._optimiser.set_population_size(population_size)  # nCPU
                opt.set_max_iterations(None)            
                opt.set_parallel(True) #72  36
                opt.set_max_unchanged_iterations(iterations=max_unchanged_iter, threshold=unchanged_iter_threshold)  # iterations=200, threshold=1e-11                
#                 opt.optimiser().set_local_global_balance(0.5)
                #opt.optimiser().set_learning_rate(0.001)
                #opt.set_population_size(population_size=80)
                print("sigma0 :", opt._optimiser._sigma0)
                print("max_iterations :", opt.max_iterations)
                print("threshold :", opt.threshold)
                                
                # Run optimisation
                transformed_params, params, logposteriors = [], [], []
                try:
                    with np.errstate(all='ignore'):
                        # Tell numpy not to issue warnings
                        transformed_p, s = opt.run()                        
                        p = transform_to_original(transformed_p)
                        transformed_params.append(transformed_p)
                        params.append(p)                        
                        logposteriors.append(s)
                        print('Found solution:          True parameters:          Old parameters:' )
                        for k, x in enumerate(p):
                            print(pints.strfloat(x) + '    ' + \
                                    pints.strfloat(real_params[k]) + '    ' + \
                                        pints.strfloat(priorparams[k]))
                except ValueError:
                    import traceback
                    traceback.print_exc()
                
                mse = MSE(transformed_params[0], transformed_real_params, axis=0)
                
                end_time = time.time()-start_time
                print(transformed_params)
                print(params)
                print(logposteriors)                
                print(opt.update_record)
                
                savedir = 'Results-' + file_name + '-staircaseramp-solution-CMAES-' + str(fit_seed)
                if not os.path.isdir(savedir):
                    os.makedirs(savedir)
                params_file_name = cell + '_%s_p%d_u%d_ui%s'%(init_param, population_size, max_unchanged_iter, str(unchanged_iter_threshold)[-2:]) + f"_nr{noise_remove}" + '.txt'
                
#                 np.savetxt(savedir + params_file_name, params[0]) # headers
                f = open(os.path.join(savedir, params_file_name), 'w')
                f.write("Cell : %s\n"%(cell))
                f.write("Initial : %s\n"%(init_param))       
                f.write("population_size : %d\n"%(population_size))       
                f.write("max_unchanged_iter : %d\n"%(max_unchanged_iter))                   
                f.write("unchanged_iter_threshold : %e\n"%(unchanged_iter_threshold))    
                if len(opt.update_record)!=0:
                    f.write("Epoch : %d  |  nUpdate : %d\n"%(opt.update_record[-1][0], opt.update_record[-1][1]))     
                else:
                    f.write("Epoch : 0  |  nUpdate : 0\n")     
                f.write("%e\n%e\n%e\n%e\n%e\n%e\n%e\n%e\n%e\n"%(params[0][0], params[0][1], params[0][2], params[0][3], params[0][4], params[0][5], params[0][6], params[0][7], params[0][8]))                
                f.write("MSE_for_param : %e\n"%(mse))
                f.write("best_score : %e\n"%(logposteriors[0]))
                f.write("iter : %d\n"%opt.iterations())
                f.write("Time : %f\n"%(opt.time()))                
                f.close()          
                
                print("--- %s seconds ---"%(end_time))                
                print("MSE_for_param : %f"%(mse))
                print("best_score : %f"%(logposteriors[0]))
                print("iter : %d"%opt.iterations())
                print("Time : %f"%(opt.time()))              
                print("="*100)
                print("="*100)
                print("="*100)
                                
                
                if False:                  
                    # Load model
                    ion_model = m.Model('../../../mmt-model-files/kylie-2017-IKr.mmt',
                                    protocol_def=protocol_def,
                                    temperature=273.15 + temperature,  # K
                                    transform=None,
                                    useFilterCap=None)  # ignore capacitive spike
#                     matplotlib.use('TkAgg')

                    real_sim_current = ion_model.simulate(np.array(real_params), times)    
                    predicted_current = ion_model.simulate(params[0], times)
                        
                    fig, ax = plt.subplots(1,1, figsize=(15,6))
                    ax.plot(times, data, label='real')
                    # axs[0].set_xlim(0,2)
                    # axs[0].set_xlabel('Time [s]', fontsize=18)
                    ax.set_ylabel('Current [mV]', fontsize=14)
                    ax.grid(True)

                    ax.plot(times, real_sim_current, label='real_sim')
                    # axs[0].set_xlim(0,2)
                    # axs[0].set_xlabel('Time [s]', fontsize=18)
                    ax.set_ylabel('Current [mV]', fontsize=14)
                    ax.grid(True)

                    ax.plot(times, predicted_current, label='predicted')
                    # axs[1].set_xlim(0,2)
                    ax.set_xlabel('Time [s]', fontsize=14)
                    ax.set_ylabel('Current [pA]', fontsize=14)
                    ax.grid(True)

                    fig.tight_layout()
                    plt.legend()
                    plt.show()

Using seed:  542811797
The number of process : 72
dt : 0.0002
Parameter fitting for A06
Estimated noise level:  10.767988932662263
sigma0:  None
(77000,)
(77000,)
Real parameters : [3.11895057e+04 4.73122577e-02 1.01732564e+02 2.96928141e-02
 5.05806145e+01 9.43549091e+01 1.75891566e+01 6.57670440e+00
 3.16275334e+01]
-----prior---------------
x0 : [1.95996752e+05 2.96501898e-01 6.59164494e+01 4.62717921e-02
 4.97958442e+01 1.07192891e+02 7.96790797e+00 6.54624352e+00
 3.14264169e+01]
transformed x0 : [0.39187188 0.34720275 0.1716572  0.26653163 0.25935297 0.6030166
 0.02074951 0.48159922 0.16367882]
MSE : 3017936661.247244
MSE for norm : 0.013865460767397673
Starting point :  [1.95996752e+05 2.96501898e-01 6.59164494e+01 4.62717921e-02
 4.97958442e+01 1.07192891e+02 7.96790797e+00 6.54624352e+00
 3.14264169e+01]
Starting transformed point :  [0.39187188 0.34720275 0.1716572  0.26653163 0.25935297 0.6030166
 0.02074951 0.48159922 0.16367882]
Starting eval_metric :  -93370388.70865336
s

In [11]:
print("천재")

천재
